## Analyzing borrowers’ risk of defaulting

Your project is to prepare a report for a bank’s loan division. You’ll need to find out if a customer’s marital status and number of children has an impact on whether they will default on a loan. The bank already has some data on customers’ credit worthiness.

Your report will be considered when building a **credit scoring** of a potential customer. A ** credit scoring ** is used to evaluate the ability of a potential borrower to repay their loan.

### Step 1. Open the data file and have a look at the general information. 

In [1]:
#use read_csv() function to read the file and the functions head(),info() to look at the general information.
import pandas as pd

credit_scoring = pd.read_csv('/datasets/credit_scoring_eng.csv')
credit_scoring.head(10)
#credit_scoring.info()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


### Conclusion

Looking at the general information I notice that some data in 'days_employed' column is ambiguous and need to be changed . The number of days a customer has been working can't be negative and we also should have integer numbers there. 

### Step 2. Data preprocessing

### Processing missing values

In [2]:
#finding NaN values
percentage_missing = (credit_scoring.isnull().sum())/len(credit_scoring)
print("Percentage of missing values :")
print('')
print(percentage_missing)

#take the two columns with missing values in a variable to process them
missing_values =['days_employed','total_income']

#take a look at the income_type column where days_employed and total_income values are missing
credit_scoring[credit_scoring['days_employed'].isnull()]['income_type'].value_counts()
credit_scoring[credit_scoring['total_income'].isnull()]['income_type'].value_counts()
#as I see that a lot of employees have missing values I assumed that there are some mistakes and decide to calculate the median value for the income_type 'employed' and fill the missing values with it
medianDaysEmployed = credit_scoring[credit_scoring['income_type']=='employee']['days_employed'].median()
medianTotalIncome = credit_scoring[credit_scoring['income_type']=='employee']['total_income'].median()

#filling with median
credit_scoring ['days_employed'] = credit_scoring['days_employed'].fillna(medianDaysEmployed)
credit_scoring ['total_income'] = credit_scoring['total_income'].fillna(medianTotalIncome)
#check for missing values
credit_scoring['days_employed'].isnull().sum()
credit_scoring['total_income'].isnull().sum()

#fill the ' 0 ' age with a valid number
credit_scoring[credit_scoring['dob_years']==0]['income_type'].value_counts()
medianEmployed = credit_scoring[credit_scoring['income_type']=='employee']['dob_years'].median()
#replace age '0' for employed with the median value
credit_scoring.loc[(credit_scoring['income_type']=='employee')&(credit_scoring['dob_years']==0),'dob_years']=medianEmployed
credit_scoring[credit_scoring['dob_years']==0]['income_type'].value_counts()

#replace age '0' for retiree with the median value
medianRetiree = credit_scoring[credit_scoring['income_type']=='retiree']['dob_years'].median()
credit_scoring.loc[(credit_scoring['income_type']=='retiree')&(credit_scoring['dob_years']==0),'dob_years']=medianRetiree
credit_scoring[credit_scoring['dob_years']==0]['income_type'].value_counts()

#replace age '0' for business with the median value
medianBusiness = credit_scoring[credit_scoring['income_type']=='business']['dob_years'].median()
credit_scoring.loc[(credit_scoring['income_type']=='business')&(credit_scoring['dob_years']==0),'dob_years']=medianBusiness
credit_scoring[credit_scoring['dob_years']==0]['income_type'].value_counts()

#replace age '0' for civil servant with the median value
medianCivil = credit_scoring[credit_scoring['income_type']=='civil servant']['dob_years'].median()
credit_scoring.loc[(credit_scoring['income_type']=='civil servant')&(credit_scoring['dob_years']==0),'dob_years']=medianCivil
credit_scoring[credit_scoring['dob_years']==0]['income_type'].value_counts()

Percentage of missing values :

children            0.000000
days_employed       0.100999
dob_years           0.000000
education           0.000000
education_id        0.000000
family_status       0.000000
family_status_id    0.000000
gender              0.000000
income_type         0.000000
debt                0.000000
total_income        0.100999
purpose             0.000000
dtype: float64


Series([], Name: income_type, dtype: int64)

 Conclusion

 I used isnull() and sum() functions to find the missing values and since days_employed and total_income have missing values I thought it could be a misspelling and decided to fill them with the median. .

### Data type replacement

In [3]:
credit_scoring.info()
#change the data type for days_employed column from float to int
credit_scoring['days_employed'] = credit_scoring ['days_employed'].astype(int)
credit_scoring['days_employed'].value_counts()

#change the negative numbers with the median of the positive numbers 
medianDaysEmployed=credit_scoring[credit_scoring['days_employed']>0]['days_employed'].median()
credit_scoring.loc[credit_scoring['days_employed']<0,'days_employed']=medianDaysEmployed
credit_scoring[credit_scoring['days_employed']<0]['days_employed'].sum()

medianTotalValues=credit_scoring[credit_scoring['total_income']>0]['total_income'].median()
credit_scoring.loc[credit_scoring['total_income']<0,'total_income']=medianTotalValues
credit_scoring[credit_scoring['total_income']<0]['total_income'].sum()

#look at the debt depending on the number of children 
credit_scoring.groupby('children')['debt'].value_counts() # we see -1
credit_scoring.loc[credit_scoring['children'] == -1,'debt'] = 1#replace with 1
 
credit_scoring.groupby('family_status')['debt'].value_counts()

credit_scoring['differenceChild'] =credit_scoring.loc[credit_scoring['debt']==0,'children'].value_counts()- credit_scoring[credit_scoring['debt']==1]['children'].value_counts()
credit_scoring['differenceFamilyStatus'] = credit_scoring.loc[credit_scoring['debt']==0]['family_status_id'].value_counts()- credit_scoring[credit_scoring['debt']==1]['family_status_id'].value_counts()

#group the columns to see how much having childs and family status have an impact on the debt
pivot_table_child = credit_scoring.pivot_table(index='children', columns='debt', values='differenceChild',aggfunc='mean')
pivot_table_family = credit_scoring.pivot_table(index='debt', columns='family_status_id', values='differenceFamilyStatus',aggfunc='mean')
 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  float64
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(3), int64(4), object(5)
memory usage: 2.0+ MB


### Conclusion

Changing the data type from float to int and fixing the mistakes from 'children' column.

### Processing duplicates

In [4]:
#look for duplicates
credit_scoring.duplicated().sum()

#I think that  duplicates should not be deleted so I just decide to process the education column for a better view 

credit_scoring['education']=credit_scoring['education'].str.lower()
credit_scoring['education'].value_counts()

secondary education    15233
bachelor's degree       5260
some college             744
primary education        282
graduate degree            6
Name: education, dtype: int64

I just think that deleting duplicates is not a good approch because I lose a lot of data information so I just decided to work on the education column. 

### Categorizing Data

In [5]:
credit_scoring['purpose'].value_counts()
#categorize data from the 'purpose' column

credit_scoring.loc[credit_scoring['purpose']=='to have a wedding','purpose']='wedding'
credit_scoring.loc[credit_scoring['purpose']=='having a wedding','purpose']='wedding'
credit_scoring.loc[credit_scoring['purpose']=='wedding ceremony','purpose']='wedding'

credit_scoring.loc[credit_scoring['purpose']=='transactions with commercial real estate','purpose']='real estate transactions'
credit_scoring.loc[credit_scoring['purpose']=='transactions with my real estate','purpose']='real estate transactions'
credit_scoring.loc[credit_scoring['purpose']=='buy commercial real estate','purpose']='real estate'
credit_scoring.loc[credit_scoring['purpose']=='housing transactions','purpose']='real estate transactions'
credit_scoring.loc[credit_scoring['purpose']=='buying property for renting out','purpose']='real estate transactions'
credit_scoring.loc[credit_scoring['purpose']=='purchase of the house','purpose']='real estate transactions'
credit_scoring.loc[credit_scoring['purpose']=='purchase of the house for my family','purpose']='real estate transactions'
credit_scoring.loc[credit_scoring['purpose']=='buy real estate','purpose']='real estate transactions'
credit_scoring.loc[credit_scoring['purpose']=='purchase of my own house','purpose']='real estate transactions'
credit_scoring.loc[credit_scoring['purpose']=='buy residential real estate','purpose']='real estate transactions'

credit_scoring.loc[credit_scoring['purpose']=='housing','purpose']='building a real estate'
credit_scoring.loc[credit_scoring['purpose']=='construction of own property','purpose']='building a real estate'
credit_scoring.loc[credit_scoring['purpose']=='property','purpose']='building a real estate'
credit_scoring.loc[credit_scoring['purpose']=='building a property','purpose']='building a real estate'
credit_scoring.loc[credit_scoring['purpose']=='housing renovation','purpose']='building a real estate'

credit_scoring.loc[credit_scoring['purpose']=='going to university','purpose']='education'
credit_scoring.loc[credit_scoring['purpose']=='supplementary education','purpose']='education'
credit_scoring.loc[credit_scoring['purpose']=='university education','purpose']='education'
credit_scoring.loc[credit_scoring['purpose']=='to get a supplementary education','purpose']='education'
credit_scoring.loc[credit_scoring['purpose']=='getting an education','purpose']='education'
credit_scoring.loc[credit_scoring['purpose']=='profile education','purpose']='education'
credit_scoring.loc[credit_scoring['purpose']=='getting higher education','purpose']='education'
credit_scoring.loc[credit_scoring['purpose']=='to become educated','purpose']='education'

credit_scoring.loc[credit_scoring['purpose']=='buying my own car','purpose']='purchase of a car'
credit_scoring.loc[credit_scoring['purpose']=='car','purpose']='purchase of a car'
credit_scoring.loc[credit_scoring['purpose']=='second-hand car purchase','purpose']='purchase of a car'
credit_scoring.loc[credit_scoring['purpose']=='to own a car','purpose']='purchase of a car'
credit_scoring.loc[credit_scoring['purpose']=='buying a second-hand car','purpose']='purchase of a car'
credit_scoring.loc[credit_scoring['purpose']=='cars','purpose']='purchase of a car'
credit_scoring.loc[credit_scoring['purpose']=='to buy a car','purpose']='purchase of a car'
credit_scoring.loc[credit_scoring['purpose']=='car purchase','purpose']='purchase of a car'
credit_scoring.loc[credit_scoring['purpose']=='car','purpose']='purchase of a car'
credit_scoring.loc[credit_scoring['purpose']=='car','purpose']='purchase of a car'
credit_scoring.loc[credit_scoring['purpose']=='car','purpose']='purchase of a car'
credit_scoring.loc[credit_scoring['purpose']=='car','purpose']='purchase of a car'

credit_scoring['purpose'].value_counts()

credit_scoring['children'].value_counts()
credit_scoring.loc[credit_scoring['children'] == -1,'children'] = 1
credit_scoring.loc[credit_scoring['children'] == 20,'children'] = 2
credit_scoring['children'].value_counts()


def having_children(value):
    
    if value == 0 :
        return 'no children'
    if value > 2:
        return 'have more than two children'
    return 'have children'
    

credit_scoring['children_status'] = credit_scoring['children'].apply(having_children)
print(credit_scoring['children_status'].value_counts())
print()

def income_status(total_income):
    
    if total_income <= 15000:
        return 'Low income'
    if total_income <= 25000:
        return 'Average income'
    if total_income <= 35000:
        return 'High income'
    return 'Very high income'
 
credit_scoring['income_status'] = credit_scoring['total_income'].apply(income_status)
print(credit_scoring['income_status'].value_counts())


no children                    14149
have children                   6996
have more than two children      380
Name: children_status, dtype: int64

Average income      9179
High income         4535
Very high income    4068
Low income          3743
Name: income_status, dtype: int64


### Conclusion

 After spending some time trying to find an easier way to categorize the 'purpose' column, I just gave up and do it this way.

### Step 3. Answer these questions

- Is there a relation between having kids and repaying a loan on time?

In [6]:
#pivot_table_child
percentage_repay = (credit_scoring[credit_scoring['debt']==0]['children_status'].value_counts())/len(credit_scoring)
print("Percentage for repaying the loan on time :")
print('')
print(percentage_repay)
percentage_not_repay = (credit_scoring[credit_scoring['debt']==1]['children_status'].value_counts())/len(credit_scoring)
print("Percentage for defaulting on the loan :")
print('')
print(percentage_not_repay)

Percentage for repaying the loan on time :

no children                    0.607944
have children                  0.292822
have more than two children    0.016214
Name: children_status, dtype: float64
Percentage for defaulting on the loan :

no children                    0.049384
have children                  0.032195
have more than two children    0.001440
Name: children_status, dtype: float64


### Conclusion

Borrowers with less than _3 kids_ are more likely to repay the loan on time.

- Is there a relation between marital status and repaying a loan on time?

In [7]:
pivot_table_family

family_status_id,0,1
debt,,
0,3924.0,2255.0


### Conclusion

People with marital and civil partnership status tend to repay the loan on time.

- Is there a relation between income level and repaying a loan on time?

In [8]:
credit_scoring[credit_scoring['debt']==1]['total_income'].median()
credit_scoring[credit_scoring['debt']==0]['total_income'].median()

credit_scoring[credit_scoring['debt']==1]['total_income'].mean()
credit_scoring[credit_scoring['debt']==0]['total_income'].mean()

26444.571974490827

### Conclusion

Looks like repaying a loan on time does not depend on the income level.

- How do different loan purposes affect on-time repayment of the loan?

In [9]:
credit_scoring.groupby('purpose')['debt'].value_counts()

purpose                   debt
building a real estate    0       3500
                          1        274
education                 0       3642
                          1        380
purchase of a car         0       3902
                          1        413
real estate               0        617
                          1         47
real estate transactions  0       5917
                          1        485
wedding                   0       2160
                          1        188
Name: debt, dtype: int64

### Conclusion

Looks like borrowers with 'real estate transactions' purposes are the best candidates.

### Step 4. General conclusion

Going through the raw data I realised that preprocessing it seems to have a big role before trying to work on data. Having wrong data types and missing values can affect out predictions and categorising data is very helpful for a better view of the data.